In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import datetime as dt

In [2]:
# Initial data set up

#pd.options.display.float_format = '{:20, 2f}'.format
pd.set_option('display.max_columns', 999)
pd.options.mode.chained_assignment = None

In [3]:
# Load the data

df = pd.read_csv('online_sales_dataset.csv')

In [4]:
df.head()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00,1.71,37039.0,Australia,0.470000,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00,41.25,19144.0,Spain,0.190000,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium
2,231932,SKU_1501,Headphones,49,2020-01-01 02:00,29.11,50472.0,Germany,0.350000,Bank Transfer,23.03,Electronics,Online,Returned,UPS,Berlin,High
3,465838,SKU_1760,Desk Lamp,14,2020-01-01 03:00,76.68,96586.0,Netherlands,0.140000,paypall,11.08,Accessories,Online,Not Returned,Royal Mail,Rome,Low
4,359178,SKU_1386,USB Cable,-30,2020-01-01 04:00,-68.11,NaN,United Kingdom,1.501433,Bank Transfer,NaN,Electronics,In-store,Not Returned,FedEx,NaN,Medium


In [5]:
df_raw = df.copy()

In [6]:
# Change dtypes for columns

df_raw['InvoiceNo'] = df_raw['InvoiceNo'].astype(str) 

df_raw['CustomerID'] = df_raw['CustomerID'].astype(str) 

df_raw['InvoiceDate'] = pd.to_datetime(df_raw['InvoiceDate'])

In [7]:
#  Format currency columns to two decimal places

df_raw['Discount'] = df_raw['Discount'].round(2)
df_raw['UnitPrice'] = df_raw['UnitPrice'].round(2)
df_raw['ShippingCost'] = df_raw['ShippingCost'].round(2)



In [8]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49782 entries, 0 to 49781
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   InvoiceNo          49782 non-null  object        
 1   StockCode          49782 non-null  object        
 2   Description        49782 non-null  object        
 3   Quantity           49782 non-null  int64         
 4   InvoiceDate        49782 non-null  datetime64[ns]
 5   UnitPrice          49782 non-null  float64       
 6   CustomerID         49782 non-null  object        
 7   Country            49782 non-null  object        
 8   Discount           49782 non-null  float64       
 9   PaymentMethod      49782 non-null  object        
 10  ShippingCost       47293 non-null  float64       
 11  Category           49782 non-null  object        
 12  SalesChannel       49782 non-null  object        
 13  ReturnStatus       49782 non-null  object        
 14  Shipme

In [9]:
df_cleaned = df_raw.copy()

In [10]:
df_cleaned.shape[0]

49782

In [11]:
df_cleaned.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,SKU_1964,White Mug,38,2020-01-01 00:00:00,1.71,37039.0,Australia,0.47,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,SKU_1241,White Mug,18,2020-01-01 01:00:00,41.25,19144.0,Spain,0.19,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium


In [12]:
# There are entries with string nan in CustomerID, 
# Find them and get a count


nan_entries = df_cleaned[df_cleaned['CustomerID'].str.contains('^nan')]

nan_count = nan_entries.shape[0]

print(f"Number of 'nan' string entries in 'CustomerID': {nan_count}")

Number of 'nan' string entries in 'CustomerID': 4978


In [13]:
# nan values as the CustomerID is of no value in this exercise so 
# remove them consider putting them in their own df to see if there 
# is some take away that can be gleaned

# Convert 'nan' strings to actual NaN values
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].replace('nan', pd.NA)

# Drop rows where 'CustomerID' is NaN
df_cleaned = df_cleaned.dropna(subset=['CustomerID'])

print("Cleaned DataFrame:\n", df_cleaned)


Cleaned DataFrame:
       InvoiceNo StockCode   Description  Quantity         InvoiceDate  \
0        221958  SKU_1964     White Mug        38 2020-01-01 00:00:00   
1        771155  SKU_1241     White Mug        18 2020-01-01 01:00:00   
2        231932  SKU_1501    Headphones        49 2020-01-01 02:00:00   
3        465838  SKU_1760     Desk Lamp        14 2020-01-01 03:00:00   
5        744167  SKU_1006  Office Chair        47 2020-01-01 05:00:00   
...         ...       ...           ...       ...                 ...   
49777    354083  SKU_1562      Blue Pen        25 2025-09-05 01:00:00   
49778    296698  SKU_1930     USB Cable         7 2025-09-05 02:00:00   
49779    177622  SKU_1766  Office Chair        43 2025-09-05 03:00:00   
49780    701213  SKU_1602      Notebook        48 2025-09-05 04:00:00   
49781    772215  SKU_1832     White Mug        30 2025-09-05 05:00:00   

       UnitPrice CustomerID         Country  Discount  PaymentMethod  \
0           1.71    37039.0    

In [14]:
# Check the number of NaN entries in the 'CustomerID' column is zero

# df_cleaned[df_cleaned['CustomerID'].isna()]

In [15]:
# Reset the index for the df

df_cleaned.reset_index(drop=True, inplace=True)

In [16]:
# Trim the SKU- off the StockCode

df_cleaned['StockCode'] = df_cleaned['StockCode'].str.replace(r'^SKU_', '', regex=True) 

In [17]:
df_cleaned.dtypes

InvoiceNo                    object
StockCode                    object
Description                  object
Quantity                      int64
InvoiceDate          datetime64[ns]
UnitPrice                   float64
CustomerID                   object
Country                      object
Discount                    float64
PaymentMethod                object
ShippingCost                float64
Category                     object
SalesChannel                 object
ReturnStatus                 object
ShipmentProvider             object
WarehouseLocation            object
OrderPriority                object
dtype: object

In [18]:
# Convert CustomerID to string 

df_cleaned['CustomerID'] = df_cleaned['CustomerID'].astype(str) 

# Remove the last two characters from CustomerID 
df_cleaned['CustomerID'] = df_cleaned['CustomerID'].str.slice(0, -2)

In [19]:
df_cleaned.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority
0,221958,1964,White Mug,38,2020-01-01 00:00:00,1.71,37039,Australia,0.47,Bank Transfer,10.79,Apparel,In-store,Not Returned,UPS,London,Medium
1,771155,1241,White Mug,18,2020-01-01 01:00:00,41.25,19144,Spain,0.19,paypall,9.51,Electronics,Online,Not Returned,UPS,Rome,Medium


In [20]:
df_cleaned['PaymentMethod'].unique()

array(['Bank Transfer', 'paypall', 'Credit Card'], dtype=object)

In [21]:
# Replace paypall with correct spelling 'paypal'

df_cleaned['PaymentMethod'] = df_cleaned['PaymentMethod'].str.replace('paypall', 'paypal')

In [22]:
df_cleaned['WarehouseLocation'].unique()

array(['London', 'Rome', 'Berlin', 'Amsterdam', 'Paris'], dtype=object)

In [23]:
df_cleaned['ReturnStatus'].unique()

array(['Not Returned', 'Returned'], dtype=object)

In [24]:
# Check for negative quantities - returns

df_cleaned[df_cleaned['Quantity'] < 0].head(10)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Discount,PaymentMethod,ShippingCost,Category,SalesChannel,ReturnStatus,ShipmentProvider,WarehouseLocation,OrderPriority


In [25]:
# Check that price is not negative or  = to 0.00

df_cleaned[df_cleaned['UnitPrice'] == 0].head(10)

df_cleaned['UnitPrice'].min()

np.float64(1.0)

In [26]:
# Combine all of the costs into one net cost

df_cleaned['NetCost'] = df_cleaned['UnitPrice'] + df_cleaned['Discount'] + df_cleaned['ShippingCost']

In [27]:
# Multiply the Quantity by the Net Cost to gert the total cost of 
# the transaction 

df_cleaned['TotalCost'] = df_cleaned['Quantity'] * df_cleaned['NetCost']

In [28]:
# Percentage of original date remaining after cleaning

len(df_cleaned) / len(df)

0.9000040175163714

In [29]:
# Remove Returns and work only with Not Returned txns

df_nr = df_cleaned[df_cleaned['ReturnStatus'] == 'Not Returned']

df_nr.reset_index(drop=True, inplace=True)

In [30]:
# Removing the returns and working only with non-return txns 
# leaves 81.2% of the data for the analysis

len(df_nr) / len(df)

0.8118797959101683

In [31]:
df_nr.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country', 'Discount', 'PaymentMethod',
       'ShippingCost', 'Category', 'SalesChannel', 'ReturnStatus',
       'ShipmentProvider', 'WarehouseLocation', 'OrderPriority', 'NetCost',
       'TotalCost'],
      dtype='object')

In [32]:
# Create a new df for the RFM and exclude unnecessary columns

selected_columns = ['CustomerID', 'InvoiceNo', 'InvoiceDate',
                    'TotalCost']

df_working = df_nr[selected_columns]

In [33]:
df_working.head(2)

,CustomerID,InvoiceNo,InvoiceDate,TotalCost
0,37039,221958,2020-01-01 00:00:00,492.86
1,19144,771155,2020-01-01 01:00:00,917.10


In [39]:
# Rename the columns to less wordy options

df_working.rename(columns= {
    'CustomerID': 'customer',
    'InvoiceNo': 'invoice',
    'InvoiceDate': 'date',
    'TotalCost': 'total'
},inplace=True)

In [40]:
df_working.head(3)

,customer,invoice,date,total
0,37039,221958,2020-01-01 00:00:00,492.86
1,19144,771155,2020-01-01 01:00:00,917.10
2,96586,465838,2020-01-01 03:00:00,1230.60


In [41]:
# Create new df to create the RFM features

df_agg = df_working.groupby(by='customer', as_index=False) \
    .agg(
    value = ('total', 'sum'),
    freq = ('total', 'nunique'),
    last = ('date', 'max')
)


In [42]:
df_agg

,customer,value,freq,last
0,10001,2607.09,1,2023-08-26 03:00:00
1,10003,432.65,1,2020-08-20 18:00:00
2,10005,4220.12,2,2024-12-29 01:00:00
3,10008,177.70,1,2025-04-13 09:00:00
4,10009,1598.40,1,2020-09-06 03:00:00
...,...,...,...,...
32601,99986,3425.36,2,2023-09-11 06:00:00
32602,99989,567.82,1,2025-08-07 01:00:00
32603,99993,913.38,1,2024-01-19 04:00:00
32604,99997,3156.00,1,2021-11-01 04:00:00
